In [1]:
import azureml
import json
from azureml.core.model import Model
from azureml.core import Workspace, Run, Datastore, Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import BatchAiCompute
from azureml.core.compute import ComputeTarget
from azureml.train.dnn import PyTorch
from azureml.train.widgets import RunDetails

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  0.1.68


In [2]:
# use this code to set up config file
#subscription_id ='<SUB_ID>'
#resource_group ='<RESOURCE_GROUP>'
#workspace_name = '<WORKSPACE>'

#try:
#   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#   ws.write_config()
#   print('Workspace configuration succeeded. You are all set!')
#except:
#   print('Workspace not found. TOO MANY ISSUES!!!')
ws = Workspace.from_config()

Found the config file in: C:\projects\PyTorch-YOLOv3\aml_config\config.json


# Managing Datastores
Register / reference datastores

In [3]:
# Register datastore
#ds = Datastore.register_azure_blob_container(workspace=ws,
#                                             datastore_name='coco',
#                                             container_name='coco',
#                                             account_name='robotdatasets',
#                                             account_key='<KEY>',
#                                             create_if_not_exists=False)
datastores = ws.datastores()
for name, ds in datastores.items():
    print(name, ds.datastore_type)

workspaceblobstore AzureBlob
workspacefilestore AzureFile
images_datastore AzureBlob
coco AzureBlob


In [4]:
ds = Datastore.get(ws, datastore_name='coco')

# Checking Compute
Manage / reference compute

In [5]:
ws.compute_targets()

{'RobotArmy': <azureml.core.compute.batchai.BatchAiCompute at 0x1baa1bd02b0>,
 'Halgpu': <azureml.core.compute.batchai.BatchAiCompute at 0x1baa1bd0080>,
 'mydsvm': <azureml.core.compute.dsvm.DsvmCompute at 0x1baa1bd0160>,
 'my-aks-9': <azureml.core.compute.aks.AksCompute at 0x1baa1bd0208>,
 'gpu-cluster': <azureml.core.compute.batchai.BatchAiCompute at 0x1baa1bd0278>}

In [6]:
compute_target = ws.compute_targets()['gpu-cluster']

# Running Experiment
Create estimator and submit job

In [7]:
# loading up pip packages
packages = open('requirements.txt', 'r').read().split("\n")[:-1]
packages

['scikit-image',
 'numpy',
 'torch>=0.4.0',
 'torchvision',
 'pillow',
 'matplotlib']

In [22]:
script_params = {
    '--base_path': ds.as_mount(), # make sure we don't download
    '--use_cuda': 'true'
}
yoloEstimator = PyTorch(source_directory='./',
                        script_params=script_params,
                        compute_target=compute_target,
                        entry_script='train.py',
                        pip_packages=packages,
                        use_gpu=True)

In [ ]:
exp = Experiment(workspace=ws, name='YOLOv3')
run = exp.submit(yoloEstimator)
run

In [ ]:
RunDetails(run).show()